# Movielens 영화 추천 실습

MF 모델 학습 방법을 토대로 영화 추천 시스템 제작

추천 시스템의 MNIST인 Movielens 데이터 사용

- 유저가 영화에 평점을 매긴 데이터가 데이터 크기 별로 있다. `MovieLens 1M Dataset` 사용 권장
- 별점 데이터는 대표적인 explicit 데이터, implicit 데이터로 간주하고 테스트해볼 수 있다.
- 별점을 시청횟수로 해석해서 생각
- 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 사용하는 컬럼만 남기기
using_cols = ['user_id', 'movie_id', 'ratings']
ratings = ratings[using_cols]
ratings.head(10)

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [3]:
# 3점 이상만 남긴다

ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f"original_data_size : {orginal_data_size}, filtered_data_size : {filtered_data_size}")
print(f"Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}")

original_data_size : 1000209, filtered_data_size : 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꾼다
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
# 쉬운 검색을 위해 문자열 소문자로 변경
movies['title'] = movies['title'].str.lower()
movies['genre'] = movies['genre'].str.lower()
movies.head(10)

,movie_id,title,genre
0,1,toy story (1995),animation|children's|comedy
1,2,jumanji (1995),adventure|children's|fantasy
2,3,grumpier old men (1995),comedy|romance
3,4,waiting to exhale (1995),comedy|drama
4,5,father of the bride part ii (1995),comedy
5,6,heat (1995),action|crime|thriller
6,7,sabrina (1995),comedy|romance
7,8,tom and huck (1995),adventure|children's
8,9,sudden death (1995),action
9,10,goldeneye (1995),action|adventure|thriller


## 데이터 탐색
- 유저 수, 영화 수, 인기 많은 영화

In [8]:
# ratings, movies 병합
data = pd.merge(ratings, movies)
data.head(10)

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),drama
1,2,1193,5,one flew over the cuckoo's nest (1975),drama
2,12,1193,4,one flew over the cuckoo's nest (1975),drama
3,15,1193,4,one flew over the cuckoo's nest (1975),drama
4,17,1193,5,one flew over the cuckoo's nest (1975),drama
5,18,1193,4,one flew over the cuckoo's nest (1975),drama
6,19,1193,5,one flew over the cuckoo's nest (1975),drama
7,24,1193,5,one flew over the cuckoo's nest (1975),drama
8,28,1193,3,one flew over the cuckoo's nest (1975),drama
9,33,1193,5,one flew over the cuckoo's nest (1975),drama


In [9]:
# 유저 수
data['user_id'].nunique()

6039

In [10]:
# 영화 수
data['movie_id'].nunique()

3628

#### pd.merge() 데이터프레임 병합
각 데이터프레임을 각 데이터에 존재하는 고유값을 기준으로 병합할때 사용   
`pd.merge(df_left, df_right, how='inner', on=None)`

In [11]:
# 인기 많은 영화 30개
movie_count = data.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

In [12]:
# 인기 많은 영화 장르 30개
movie_count = data.groupby('genre')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

genre
drama                               99388
comedy                              94264
comedy|drama                        36871
comedy|romance                      35888
drama|romance                       24835
action|thriller                     22675
drama|thriller                      16133
horror                              15260
thriller                            14925
action|adventure|sci-fi             14277
drama|war                           13766
action|sci-fi|thriller              11657
action|drama|war                    11316
crime|drama                         10960
action|sci-fi                       10594
action                               9930
comedy|drama|romance                 9804
action|adventure                     8744
action|drama                         8611
comedy|sci-fi                        7797
comedy|horror                        7523
animation|children's                 7461
animation|children's|musical         7237
animation|children's|comedy 

## 선호하는 영화 5가지 골라 ratings에 추가

In [13]:
my_favorite = ['lord of the rings, the (1978)', 'wizard of oz, the (1942)', 'casablanca (1942)', 'titanic (1997)', 'die hard (1988)']

# 'rok'이라는 user_id가 영화를 10회씩 들었다고 가정
my_playlist = pd.DataFrame({'user_id' : ['rok']*5, 'title' : my_favorite, 'counts' : [10]*5})

# user_id에 'rok' 데이터가 없으면 my_favorite 데이터 추가
if not data.isin({'user_id' : ['rok']})['user_id'].any():
    data = data.append(my_playlist)
    
data.tail(10)

,user_id,movie_id,counts,title,genre
836473,5851,3607.0,5,one little indian (1973),comedy|drama|western
836474,5854,3026.0,4,slaughterhouse (1987),horror
836475,5854,690.0,3,"promise, the (versprechen, das) (1994)",romance
836476,5938,2909.0,4,"five wives, three secretaries and me (1998)",documentary
836477,5948,1360.0,5,identification of a woman (identificazione di ...,drama
0,rok,NaN,10,"lord of the rings, the (1978)",NaN
1,rok,NaN,10,"wizard of oz, the (1942)",NaN
2,rok,NaN,10,casablanca (1942),NaN
3,rok,NaN,10,titanic (1997),NaN
4,rok,NaN,10,die hard (1988),NaN


## 결측치 있는 컬럼 제거

이거 안해줘서 중간에 값이 이상하게 나왔다

In [14]:
using_cols = ['user_id', 'counts', 'title']
data = data[using_cols]
data.tail(10)

,user_id,counts,title
836473,5851,5,one little indian (1973)
836474,5854,4,slaughterhouse (1987)
836475,5854,3,"promise, the (versprechen, das) (1994)"
836476,5938,4,"five wives, three secretaries and me (1998)"
836477,5948,5,identification of a woman (identificazione di ...
0,rok,10,"lord of the rings, the (1978)"
1,rok,10,"wizard of oz, the (1942)"
2,rok,10,casablanca (1942)
3,rok,10,titanic (1997)
4,rok,10,die hard (1988)


In [15]:
# 고유한 유저, 영화제목 찾아내는 코드
user_unique = data['user_id'].unique()
title_unique = data['title'].unique()

# 유저, 아티스트 indexing 하는 코드
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

In [16]:
# 인덱싱 잘 됐는지 확인
print(user_to_idx['rok'])
print(title_to_idx['titanic (1997)'])

6039
27


In [17]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = data['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,10,926
1,6039,10,3628
2,6039,10,361
3,6039,10,27


### 1회만 counts한 데이터 비율 확인

In [18]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['counts']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

0,836483
Ratio of only_one over all data is 0.00%


영화를 한번만 본 사람은 없는 것 같다.

## CSR Matrix 만들기

In [19]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_title = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape = (num_user, num_title))
csr_data

<6040x3629 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

# MF 모델 학습하기

## als_model = AlternatingLeastSquares 모델 직접 구성 훈련하기

In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 __init__ 파라미터 살펴보기

- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1과 4를 늘릴수록 학습 데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 한다.

In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=30, dtype=np.float32)

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3629x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/30 [00:00<?, ?it/s]

## 모델 학습 완료

- rok 벡터와 titanic 벡터를 어떻게 만드는지
- 두 벡터를 곱하면 어떤 값이 나오는지
확인하기

### 내가 선호하는 5가지 영화중 하나

In [24]:
# 선택한 5개의 영화 중 하나의 값
rok, titanic = user_to_idx['rok'], title_to_idx['titanic (1997)']
rok_vector, titanic_vector = als_model.user_factors[rok], als_model.item_factors[titanic]

In [25]:
rok_vector

array([-0.08260991,  0.50024956,  0.2899584 ,  0.03745965, -0.6984454 ,
        0.3570002 ,  0.00848134, -0.47829354, -0.05693963,  0.14889297,
       -0.20527227, -0.74494064, -0.49728063,  0.03605219,  0.10715085,
       -1.1104555 ,  0.20613624,  0.48797065,  0.3618209 , -0.27078357,
       -0.59227085,  0.15175064,  0.94011575, -0.16439578, -0.544866  ,
        0.16807906,  0.4555887 , -0.07091334,  0.5191497 , -0.27933624,
        0.61942774, -0.47792247, -0.12326884, -0.71748906, -0.2968584 ,
       -0.21581846, -1.1081934 ,  0.13478088, -0.2990173 , -0.41091374,
       -0.2174218 , -0.190159  , -0.2850791 , -0.87971765,  0.5130793 ,
        0.24652572,  0.13136467,  0.32800668, -0.44164497,  0.70049864,
        0.3877254 ,  0.14125344, -0.10737037,  0.4988822 , -0.35544345,
       -0.11367383,  0.5430015 , -0.26747876, -0.26305443, -0.30156296,
        0.7712116 ,  0.33706054, -0.84022325,  0.5953215 ,  0.7798872 ,
        0.2962593 , -0.15297212,  0.04704148,  0.71660227, -0.08

In [26]:
titanic_vector

array([-1.20116519e-02, -1.89928128e-03,  3.50486045e-03,  6.13449141e-03,
        2.11259499e-02,  3.42710540e-02,  2.06934428e-03, -2.65853573e-02,
       -2.64042523e-03, -1.64007824e-02, -2.45174896e-02,  1.45985857e-02,
       -3.11321653e-02,  3.18529233e-02, -8.81904177e-03, -1.45344557e-02,
        2.69001890e-02,  1.40657397e-02,  1.97727028e-02, -3.83615158e-02,
       -7.68056605e-03,  1.43438336e-02,  4.24482785e-02, -1.07082101e-02,
        1.47644838e-03,  3.06202676e-02,  3.40777226e-02,  6.83638230e-02,
        1.63488574e-02, -2.09076796e-02,  3.35681103e-02,  7.87606521e-04,
        5.17808273e-02,  3.69680040e-02,  3.25511937e-05, -1.14655085e-02,
        2.08701845e-02,  2.64225574e-03, -3.45597044e-02, -1.68021210e-02,
        3.18703279e-02, -8.23883153e-03,  2.31191013e-02, -1.77693237e-02,
        3.05214562e-02,  4.61026467e-02, -1.08846202e-02,  1.70113593e-02,
        9.12005268e-03,  5.37980981e-02, -1.58257633e-02,  3.33191790e-02,
       -3.02788205e-02,  

In [27]:
# rok과 titanic을 내적하는 코드
np.dot(rok_vector, titanic_vector)

0.9066202

처음 `factors = 100`, `iterations = 15` 값을 줬을때 0.55 정도의 애매한 값이 나왔다   
factors를 늘리거나 iterations를 늘려야 한다.  

두 번째로는 두 배인 `factors = 200`, `iterations = 30` 으로 주니 0.78 정도의 값이 나온다   
제발 과적합만 안되길

### 그 외의 영화

In [28]:
sabrina = title_to_idx['sabrina (1995)']
sabrina_vector = als_model.item_factors[sabrina]

np.dot(rok_vector, sabrina_vector)

0.009091483

내가 선호하는 타이타닉은 0.7752의 선호도가 나오고, 처음 보는 사브리나는 0.0090의 선호도가 나온다   
사브리나라는 영화가 뭔지는 모르겠지만 내가 전혀 좋아하지 않는 영화인 것 같다.


새로 돌리니까 갑자기 타이타닉 선호도가 0.9066으로 확 증가하였다........

#### 번외

다른 4개의 영화도 넣어본다

In [29]:
# 5개 중 다른 영화 값
lord_rings = title_to_idx['lord of the rings, the (1978)']
lord_rings_vector = als_model.item_factors[lord_rings]

casablanca = title_to_idx['casablanca (1942)']
casablanca_vector = als_model.item_factors[casablanca]

wizard_of_oz = title_to_idx['wizard of oz, the (1942)']
wizard_of_oz_vector = als_model.item_factors[wizard_of_oz]

die_hard = title_to_idx['die hard (1988)']
die_hard_vector = als_model.item_factors[die_hard]

In [30]:
np.dot(rok_vector, lord_rings_vector)

0.49853024

In [31]:
np.dot(rok_vector, casablanca_vector)

0.88974375

In [32]:
np.dot(rok_vector, wizard_of_oz_vector)

0.05098418

In [33]:
np.dot(rok_vector, die_hard_vector)

0.6639509

내가 선호하는 영화들을 값으로 넣었을때   

- 반지의 제왕 : 0.49853024 
- 타이타닉 : 0.9066202
- 카사블랑카 : 0.88974375
- 오즈의 마법사 : 0.05098418
- 다이하드 : 0.6639509

처음에 선호하는 영화로 지정한 영화들의 백터 내적값을 계산했는데  
이상하게 오즈의 마법사가 전혀 선호하지 않을 정도의 값이 나왔다   
왜 그런지 모르겠다.


# 비슷한 영화 찾기 + 영화 추천하기

## 비슷한 영화 찾기

In [34]:
# 처음엔 내가 추가한 die hard로 확인
favorite_movie = 'die hard (1988)'
movie_id = title_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(194, 0.9999999),
 (200, 0.5900671),
 (172, 0.49998263),
 (141, 0.43336654),
 (111, 0.42643186),
 (3628, 0.4231267),
 (865, 0.41076973),
 (57, 0.4063325),
 (131, 0.3773443),
 (651, 0.35329282),
 (120, 0.34238514),
 (199, 0.33905423),
 (3169, 0.33573824),
 (1091, 0.33524194),
 (137, 0.3347607)]

In [35]:
# 영화 Id를 영화 이름으로 매핑
# title_to_idx를 뒤집어, index로 title이름 얻는 dict 생성
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_movie]

['die hard (1988)',
 'terminator, the (1984)',
 'indiana jones and the last crusade (1989)',
 'fugitive, the (1993)',
 'die hard 2 (1990)',
 'wizard of oz, the (1942)',
 'predator (1987)',
 'untouchables, the (1987)',
 'rocky (1976)',
 'aliens (1986)',
 'raiders of the lost ark (1981)',
 'goldfinger (1964)',
 'mr. & mrs. smith (1941)',
 'no way out (1987)',
 'die hard: with a vengeance (1995)']

In [36]:
def get_similar_movie(movie_name: str):
    movie_id = title_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_title[i[0]] for i in similar_movie]
    return similar_movie

In [37]:
get_similar_movie('titanic (1997)')

['titanic (1997)',
 'wizard of oz, the (1942)',
 "you've got mail (1998)",
 'ever after: a cinderella story (1998)',
 'walking dead, the (1995)',
 'city of angels (1998)',
 'hot spot, the (1990)',
 'funeral, the (1996)',
 'saragossa manuscript, the (rekopis znaleziony w saragossie) (1965)',
 'ghost (1990)']

In [38]:
get_similar_movie('rocky (1976)')

['rocky (1976)',
 'rocky ii (1979)',
 'kramer vs. kramer (1979)',
 'rocky iii (1982)',
 'die hard (1988)',
 'french connection, the (1971)',
 'jaws (1975)',
 'rocky iv (1985)',
 'goldfinger (1964)',
 'rocky v (1990)']

In [39]:
get_similar_movie('matrix, the (1999)')

['matrix, the (1999)',
 'terminator 2: judgment day (1991)',
 'terminator, the (1984)',
 'total recall (1990)',
 'fugitive, the (1993)',
 'face/off (1997)',
 'fifth element, the (1997)',
 'running free (2000)',
 'star wars: episode v - the empire strikes back (1980)',
 'surviving picasso (1996)']

전체적으로 비슷한 장르의 영화들로 구성된 영화들이 잘 나오는 것 같다.

## 내가 가장좋아할 만한 영화 추천 받기

In [40]:
user = user_to_idx['rok']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended                                        

[(988, 0.34820744),
 (200, 0.3386907),
 (656, 0.2848193),
 (865, 0.27668938),
 (172, 0.27518222),
 (120, 0.24363217),
 (141, 0.23255563),
 (126, 0.22248152),
 (704, 0.22014204),
 (601, 0.21868485),
 (993, 0.21068604),
 (1194, 0.20786002),
 (746, 0.19234547),
 (38, 0.18727806),
 (169, 0.182957),
 (291, 0.18143693),
 (1124, 0.18056974),
 (651, 0.1793052),
 (1042, 0.17522462),
 (731, 0.16641885)]

In [41]:
[idx_to_title[i[0]] for i in movie_recommended]

['glory (1989)',
 'terminator, the (1984)',
 'african queen, the (1951)',
 'predator (1987)',
 'indiana jones and the last crusade (1989)',
 'raiders of the lost ark (1981)',
 'fugitive, the (1993)',
 'shakespeare in love (1998)',
 'tron (1982)',
 'citizen kane (1941)',
 'shining, the (1980)',
 'fatal attraction (1987)',
 'perfect storm, the (2000)',
 'sixth sense, the (1999)',
 'fish called wanda, a (1988)',
 'maltese falcon, the (1941)',
 'body heat (1981)',
 'aliens (1986)',
 'beavis and butt-head do america (1996)',
 'hoop dreams (1994)']

In [42]:
terminator = title_to_idx['terminator, the (1984)']
explain = als_model.explain(user, csr_data, itemid=terminator)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('die hard (1988)', 0.33978285608572834),
 ('casablanca (1942)', 0.019958243176262533),
 ('titanic (1997)', 0.008969564789502536),
 ('wizard of oz, the (1942)', 0.002066048847132604),
 ('lord of the rings, the (1978)', -0.03521917361774604)]

In [43]:
citizen_kane = title_to_idx['citizen kane (1941)']
explain = als_model.explain(user, csr_data, itemid=citizen_kane)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('casablanca (1942)', 0.19041058276199588),
 ('lord of the rings, the (1978)', 0.04824728061012504),
 ('titanic (1997)', 0.01239534565281674),
 ('wizard of oz, the (1942)', 0.0023458373452093536),
 ('die hard (1988)', -0.036744396059182664)]

In [44]:
sixth_sense = title_to_idx['sixth sense, the (1999)']
explain = als_model.explain(user, csr_data, itemid=sixth_sense)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('die hard (1988)', 0.1171527706298893),
 ('lord of the rings, the (1978)', 0.0652387608735027),
 ('titanic (1997)', 0.039860479524128786),
 ('wizard of oz, the (1942)', 0.0018417917279293272),
 ('casablanca (1942)', -0.04045085545643748)]

각각 
터미네이터는 다이하드에서  
시민 케인은 카사블랑카에서   
식스센스는 다이하드와 타이타닉이 가장 높은 영향을 미친 것을 알 수 있다.

# 더 다양하게 파라미터 변경해보기

### 한 3개 정도만 더 해본다, 배수로 진행

In [45]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=800, regularization=0.01, use_gpu=False, iterations=120, dtype=np.float32)

# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/120 [00:00<?, ?it/s]

In [46]:
# 선택한 5개의 영화 중 하나의 값
rok, titanic = user_to_idx['rok'], title_to_idx['titanic (1997)']
rok_vector, titanic_vector = als_model.user_factors[rok], als_model.item_factors[titanic]

# rok과 titanic을 내적하는 코드
np.dot(rok_vector, titanic_vector)

0.9939665

- factors = 200, iterations = 30, 내적값 = 0.7752002   
- factors = 400, iterations = 60, 내적값 = 0.90778905   
- factors = 800, iterations = 120, 내적값 = 0.96866477  


확실히 파라미터를 높일 수록 좋은 값이 나오는 것을 확인할 수 있다.   
그렇다면 파라미터를 변경한 이후  
선호하지 않는 영화의 선호도, 비슷한 영화 추천, 내가 가장 좋아할만한 영화 추천을 확인해보자

위 값은 처음 돌렸을때의 파라미터 변화값으로 마지막으로 전체 코드를 한번 더 돌리니 선호도 수치가 크게 상승하였다.    
- factors = 200, iterations = 30, 내적값 = 0.9066202   
- factors = 800, iterations = 120, 내적값 = 0.9939665


### 선호하지 않는 영화의 선호도 (파라미터 변경)

In [47]:
sabrina = title_to_idx['sabrina (1995)']
sabrina_vector = als_model.item_factors[sabrina]

np.dot(rok_vector, sabrina_vector)

-0.054766625

### 비슷한 영화 추천 (파라미터 변경)

In [48]:
get_similar_movie('titanic (1997)')

['titanic (1997)',
 'wizard of oz, the (1942)',
 'walking dead, the (1995)',
 'paris, france (1993)',
 'held up (2000)',
 'gay deceivers, the (1969)',
 'falling in love again (1980)',
 'carnosaur 2 (1995)',
 'full speed (1996)',
 'jerry & tom (1998)']

In [49]:
get_similar_movie('rocky (1976)')

['rocky (1976)',
 'rocky ii (1979)',
 'one little indian (1973)',
 'jerry & tom (1998)',
 'power 98 (1995)',
 'talk of angels (1998)',
 'everything relative (1996)',
 'fausto (1993)',
 'yankee zulu (1994)',
 'allnighter, the (1987)']

In [50]:
get_similar_movie('matrix, the (1999)')

['matrix, the (1999)',
 'running free (2000)',
 'second jungle book: mowgli & baloo, the (1997)',
 'theodore rex (1995)',
 'search for one-eye jimmy, the (1996)',
 'sunchaser, the (1996)',
 'love, etc. (1996)',
 'jerry & tom (1998)',
 'tough and deadly (1995)',
 'paris was a woman (1995)']

### 내가 좋아할 만한 영화 추천 (파라미터 변경)


In [51]:
user = user_to_idx['rok']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended     

[idx_to_title[i[0]] for i in movie_recommended]

['black cauldron, the (1985)',
 'henry v (1989)',
 'swiss family robinson (1960)',
 'sleeping beauty (1959)',
 'heavy metal (1981)',
 'for love of the game (1999)',
 'black hole, the (1979)',
 'ladyhawke (1985)',
 'mickey blue eyes (1999)',
 'star trek: the motion picture (1979)',
 'to catch a thief (1955)',
 'simon birch (1998)',
 'timecop (1994)',
 'shining, the (1980)',
 'keeping the faith (2000)',
 'first blood (1982)',
 'tommy boy (1995)',
 'jungle book, the (1994)',
 'predator 2 (1990)',
 'manchurian candidate, the (1962)']

#### 영화 추천의 기여도 확인 (파라미터 변경)

In [52]:
terminator = title_to_idx['terminator, the (1984)']
explain = als_model.explain(user, csr_data, itemid=terminator)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('die hard (1988)', 0.07421644215590009),
 ('titanic (1997)', 0.011322861043994402),
 ('casablanca (1942)', -0.0008763221863855586),
 ('wizard of oz, the (1942)', -0.015525224948570917),
 ('lord of the rings, the (1978)', -0.03713861647737528)]

In [53]:
sixth_sense = title_to_idx['sixth sense, the (1999)']
explain = als_model.explain(user, csr_data, itemid=sixth_sense)
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('die hard (1988)', 0.010251442536804996),
 ('titanic (1997)', 0.0097310803479195),
 ('casablanca (1942)', 0.0022592636491411064),
 ('lord of the rings, the (1978)', -0.0012185285483732374),
 ('wizard of oz, the (1942)', -0.010769794582603997)]

# 회고



#### 타이타닉

| 파라미터 변경 전 | 파라미터 변경 후 |
| :-----: | :--------: |
| wizard of oz, the (1942) | wizard of oz, the (1942) |
| you've got mail (1998) | walking dead, the (1995) |
| ever after: a cinderella story (1998) | paris, france (1993) |
| walking dead, the (1995) | held up (2000) |
| city of angels (1998) | gay deceivers, the (1969) |


#### 로키


| 파라미터 변경 전 | 파라미터 변경 후 |
| :-----: | :--------: |
| rocky ii (1979) | rocky ii (1979) |
| kramer vs. kramer (1979) | one little indian (1973) |
| rocky iii (1982) | jerry & tom (1998) |
| die hard (1988) | power 98 (1995) |
| french connection, the (1971) | talk of angels (1998) |

#### 매트릭스

| 파라미터 변경 전 | 파라미터 변경 후 |
| :-----: | :--------: |
| terminator 2: judgment day (1991) | running free (2000) |
| terminator, the (1984) | second jungle book: mowgli & baloo, the (1997) |
| total recall (1990) | theodore rex (1995) |
| fugitive, the (1993) | search for one-eye jimmy, the (1996) |
| face/off (1997) | sunchaser, the (1996) |


#### 유저 추천 영화

| 파라미터 변경 전 | 파라미터 변경 후 |
| :-----: | :--------: |
| glory (1989) | black cauldron, the (1985) |
| terminator, the (1984) | henry v (1989) |
| african queen, the (1951) | swiss family robinson (1960) |
| predator (1987) | sleeping beauty (1959) |
| indiana jones and the last crusade (1989) | heavy metal (1981) |

위 표는 파라미터를 높이기 전과 후를 비교해본 값이다

파라미터 변경 전에는 전체적으로 비슷한 영화를 추천해주는 것을 볼 수 있지만    
파라미터 변경 후에는 전혀 생뚱맞은 장르도 맞지않는 영화를 추천해준다.

이는 과적합으로 의심이 된다 데이터셋에 너무 적용되서 오히려 정확도가 더 떨어졌다

솔직히 파라미터를 높이면서 너무 높은게 아닌가 고민을 하긴 했지만 정말로 너무 별로인 결과를 알 수 있다.

그리고 파라미터를 변경하기 전에도 높은 값이 나오기 때문에 적절하다고 볼 수 있다.
